In [ ]:
pip install  geneticalgorithm

In [ ]:
from math import sin, cos
import random
import numpy as np
import matplotlib.pyplot as plt

from poisson import getTurbLoc, loadPowerCurve, binWindResourceData, preProcessing, preProcessing_greedy, getAEP, getAEP_greedy, checkConstraints

from poisson import Bridson_sampling

num_turbines = 1

# Turbine Specifications.
# -**-SHOULD NOT BE MODIFIED-**-
turb_specs    =  {   
                     'Name': 'Anon Name',
                     'Vendor': 'Anon Vendor',
                     'Type': 'Anon Type',
                     'Dia (m)': 100,
                     'Rotor Area (m2)': 7853,
                     'Hub Height (m)': 100,
                     'Cut-in Wind Speed (m/s)': 3.5,
                     'Cut-out Wind Speed (m/s)': 25,
                     'Rated Wind Speed (m/s)': 15,
                     'Rated Power (MW)': 3
                 }
turb_diam      =  turb_specs['Dia (m)']
turb_rad       =  turb_diam/2 

# Turbine x,y coordinates
turb_coords   =  getTurbLoc('turbine_loc_test.csv')

# Load the power curve
power_curve   =  loadPowerCurve('power_curve.csv')

# Pass wind data csv file location to function binWindResourceData.
# Retrieve probabilities of wind instance occurence.
wind_inst_freq =  binWindResourceData('wind_data_2009.csv')   

# Doing preprocessing to avoid the same repeating calculations. Record 
# the required data for calculations. Do that once. Data are set up (shaped)
# to assist vectorization. Used later in function totalAEP.
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing_greedy(power_curve, num_turbines)

def greedy_solution(points):
    solution = np.array([[]])
    solution = np.array([], dtype=np.int64).reshape(0,2)
    best_value = 0
    best_point = 0
    x = np.array([[]])
    for i in range(n_turbs):
        best_value = 0
        for j in range(n_turbs):
            objec = points[j]
            if(objec not in solution):
                solution = np.vstack((solution, objec))
                obj = getAEP_greedy(turb_rad, solution, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
                if(obj >= best_value):
                    best_value = obj
                    best_point = objec
                solution = np.delete(solution, len(solution) - 1, 0)
        solution = np.vstack((solution, best_point))
    neighbors = np.array(list(filter(lambda x: x not in solution, points)))
    return solution

def perturb(solution, neighbors, position, best_value):
    best_point = solution[position]
    print("Initial ", best_value)
    for i in neighbors:
        solution[position] = i
        obj = getAEP(turb_rad, solution, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
        if(obj >= best_value):
            best_value = obj
            best_point = i
    solution[position] = best_point
    print("Later ", best_value)

    return solution, best_value

def local_search(solution):
    best_solution = solution.copy()
    neighbors = np.array(list(filter(lambda x: x not in solution, points)))
    best_value = getAEP(turb_rad, solution, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
    for position in range(n_turbs):
        solution = best_solution.copy()
        improved = False
        for i in neighbors:
            solution[position] = i.copy()
            value = getAEP(turb_rad, solution, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
            if(value >= best_value):
                best_value = value
                best_point = i
                improved = True
                best_solution = solution.copy()
        neighbors = np.array(list(filter(lambda x: x not in best_solution, points)))
    return best_solution

width = 3900
height = 3900
n_turbs = 50
points = Bridson_sampling(width=width, height=height, radius=400)
vec = []
for i in points:
    i[0] = i[0] + 50
    i[1] = i[1] + 50

sol = greedy_solution(points)
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve)
solu = local_search(sol)
best_solution = getAEP(turb_rad, solu, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
print(best_solution)
checkConstraints(solu, turb_diam)


467.58244354248046
Both perimeter and proximity constraints are satisfied !!



()

In [ ]:
import pandas as pd
loc=pd.DataFrame(solu)
loc.to_csv('Check1.csv')